In [ ]:
import os

from multiprocessing.pool import ThreadPool

import ctypes
from ctypes import *

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder

from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

from params import get_aaf

In [ ]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'
    decoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_dataset.dat'
    decoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query,
                                        decoded_dataset, decoded_query]})

n = 60000

In [ ]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # decode encoded datasets
    decoded_train = autoencoder.decode(encoded_train)
    decoded_test = autoencoder.decode(encoded_test)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

    # normalize decoded datasets
    decoded_train = normalize(decoded_train)
    decoded_test = normalize(decoded_test)

    # save decoded datasets
    save_decoded_binary(decoded_train, decoded_dataset)
    save_decoded_binary(decoded_test, decoded_query)

In [ ]:
best_params_lsh = [7, 4, 1875, 0, 0.6]      # k, L, table_size, query_trick, window
best_params_hypercube = [3, 67, 1000, 0.42] # k, M, probes, window

# k, E, R
best_params_gnns = {'model_conv_46.keras':  [92, 44, 9],
                    'model_conv_12.keras':  [86, 59, 8],
                    'model_conv_19.keras':  [96, 87, 7],
                    'model_dense_26.keras': [70, 69, 9],
                    'model_dense_43.keras': [100, 66, 10], 
                    'model_dense_1.keras':  [64, 58, 10]}

# l
best_params_mrng = {'model_dense_26.keras': [883],
                    'model_conv_46.keras':  [859],
                    'model_conv_19.keras':  [507],
                    'model_dense_1.keras':  [428],
                    'model_dense_43.keras': [818], 
                    'model_conv_12.keras':  [814]}

# l, m, k, lq
best_params_nsg = {'model_dense_26.keras': [987, 52, 69, 667],
                   'model_conv_46.keras':  [987, 43, 87, 476],
                   'model_dense_43.keras': [960, 102, 86, 768],
                   'model_dense_1.keras':  [996, 11, 98, 657],
                   'model_conv_12.keras':  [635, 197, 50, 907],
                   'model_conv_19.keras':  [663, 99, 41, 533]}

In [14]:
def run_brute_force(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    conf = {
        'model': b'BRUTE',
        'vals': [],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'BRUTE', average_time.value, aaf_lat_init.value])

def run_lsh(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]
    
    conf = {
        'model': b'LSH',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'LSH', average_time.value, aaf_lat_init.value])

def run_hypercube(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]
    
    conf = {
        'model': b'CUBE',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'CUBE', average_time.value, aaf_lat_init.value])

def run_gnns(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]
    
    conf = {
        'model': b'GNNS',
        'vals': best_params_gnns[model],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'GNNS', average_time.value, aaf_lat_init.value])

def run_mrng(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]
    
    conf = {
        'model': b'MRNG',
        'vals': best_params_mrng[model],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'MRNG', average_time.value, aaf_lat_init.value])

def run_nsg(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]
    
    conf = {
        'model': b'NSG',
        'vals': best_params_nsg[model],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'NSG', average_time.value, aaf_lat_init.value])

In [ ]:
algorithms_to_functions = {'BRUTE': run_brute_force,
                           'LSH'  : run_lsh,
                           'CUBE' : run_hypercube,
                           'GNNS' : run_gnns,
                           'MRNG' : run_mrng,
                           'NSG'  : run_nsg}

pool = ThreadPool(processes=4)

rows = []
for model in models:
    for algorithm in algorithms_to_functions:
        pool.apply_async(algorithms_to_functions[algorithm], (model,))

pool.close()
pool.join()

In [ ]:
col_models, col_algorithms, col_average_time, col_aaf_lat_init = [], [], [], []

for row in rows:
    model, algorithm, average_time, aaf_lat_init = row

    col_models.append(model)
    col_algorithms.append(algorithm)
    col_average_time.append(average_time)
    col_aaf_lat_init.append(aaf_lat_init)

col_dict = {'model': col_models, 'algorithm': col_algorithms,
            'average time': col_average_time, 'AAF (latent to initial)': col_aaf_lat_init}

df = pandas.DataFrame(data=col_dict)
df

In [ ]:
df_sorted = df.copy(deep=True)
df_sorted = df_sorted.sort_values(by='model', ascending=True)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

In [ ]:
df_sorted = df.copy(deep=True)
df_sorted = df_sorted.sort_values(by='algorithm', ascending=True)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted